In [1]:
import pandas as pd

In [233]:
import numpy as np

In [234]:
crime=pd.read_csv('crime_w_zipcodes.csv',index_col=0)

C:\Apps\HarshD\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [235]:
crime.head()

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,LAW_CAT_CD,Latitude,Longitude,Lat_Lon,zipcode,neighborhood
0,876525802,BROOKLYN,04/10/2008,MISDEMEANOR,40.669,-73.913,"40.669,-73.913",11212.0,Central Brooklyn
1,402693528,MANHATTAN,06/03/2007,MISDEMEANOR,40.802,-73.946,"40.802,-73.946",10026.0,Central Harlem
2,832030069,QUEENS,02/16/2010,FELONY,40.700,-73.831,"40.7,-73.831",11418.0,Southwest Queens
3,538308771,BROOKLYN,11/10/2009,MISDEMEANOR,40.681,-73.955,"40.681,-73.955",11216.0,Central Brooklyn
5,141085517,BROOKLYN,06/23/2011,FELONY,40.678,-73.926,"40.678,-73.926",11233.0,Central Brooklyn


In [236]:
crime.LAW_CAT_CD.value_counts()

MISDEMEANOR    3678454
FELONY         1989376
VIOLATION       820185
Name: LAW_CAT_CD, dtype: int64

In [231]:
# crime['crime_type']=crime['LAW_CAT_CD']
#mapping = {'VIOLATION ': 1, 'MISDEMEANOR': 2,'FELONY':3}
#crime.replace({'crime_type': mapping},inplace=True)

In [237]:
crime.isnull().sum()

CMPLNT_NUM         0
BORO_NM            0
CMPLNT_FR_DT       0
LAW_CAT_CD         0
Latitude           0
Longitude          0
Lat_Lon            0
zipcode            0
neighborhood    8533
dtype: int64

In [239]:
#crime.dropna(inplace=true)

In [238]:
crime.dtypes

CMPLNT_NUM        int64
BORO_NM          object
CMPLNT_FR_DT     object
LAW_CAT_CD       object
Latitude        float64
Longitude       float64
Lat_Lon          object
zipcode         float64
neighborhood     object
dtype: object

In [240]:
airbnb=pd.read_csv('Airbnb_yelp_subway.csv',index_col=0)

In [241]:
airbnb.head(2)

,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,...,total_restaurants_reviews_5,restaurants_avg_rating_5,total_bars_1,total_bars_reviews_1,bars_avg_rating_1,bars_yelp_5,total_bars_reviews_5,bars_avg_rating_5,nearest_station,distance_to_station
0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,...,40022.0,4.057692,1.0,10.0,4.5,21.0,8701.0,4.000000,42nd St - Bryant Pk,0.093386
1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,295,...,4086.0,4.240741,NaN,NaN,NaN,12.0,790.0,4.208333,Classon Ave,0.417452


In [248]:
airbnb_nhood=pd.DataFrame(airbnb['neighborhood'].value_counts().reset_index())

In [249]:
airbnb_nhood.head(2)

,index,neighborhood
0,Bushwick and Williamsburg,4483
1,Central Brooklyn,4158


In [250]:
crime_nhood = pd.DataFrame(crime['neighborhood'].value_counts().reset_index())

In [251]:
crime_nhood.head(2)

,index,neighborhood
0,Central Brooklyn,367261
1,Chelsea and Clinton,295523


In [252]:
print(crime_nhood.shape)
print(airbnb_nhood.shape)

(51, 2)
(51, 2)


In [253]:
match_hoods = pd.merge(airbnb_nhood, crime_nhood,on=['index'], how='outer',suffixes=('_airbnb', '_crime'))

In [254]:
match_hoods .head(2)

,index,neighborhood_airbnb,neighborhood_crime
0,Bushwick and Williamsburg,4483.0,211233.0
1,Central Brooklyn,4158.0,367261.0


In [255]:
#Count records where there is no match on each side.
airbnb_nomatch = match_hoods.neighborhood_airbnb[match_hoods.neighborhood_crime.isnull()]
print('There are ' + str(airbnb_nomatch.sum()) + ' AirBnB listings with no match in the crime data.')

crime_nomatch = match_hoods.neighborhood_crime[match_hoods.neighborhood_airbnb.isnull()]
print('There are ' + str(crime_nomatch.sum()) + ' Crime listings with no match in the AirBnB data.')

There are 12.0 AirBnB listings with no match in the crime data.
There are 1192.0 Crime listings with no match in the AirBnB data.


In [256]:
print(match_hoods.sort_values('index'))

                            index  neighborhood_airbnb  neighborhood_crime
20                   Borough Park                589.0            120253.0
48                          Bronx                  2.0                 NaN
50                         Bronx                   2.0               499.0
34         Bronx Park and Fordham                181.0            237765.0
49                       Brooklyn                  2.0                 NaN
0       Bushwick and Williamsburg               4483.0            211233.0
28         Canarsie and Flatlands                254.0            141466.0
36                  Central Bronx                120.0            236762.0
1                Central Brooklyn               4158.0            367261.0
7                  Central Harlem               2066.0            204376.0
37                 Central Queens                105.0             38781.0
2             Chelsea and Clinton               4079.0            295523.0
15                    Eas

In [257]:
##to_drop=['Yonkers','New Rochelle']
#match_hoods[~match_hoods['neighborhood_yelp'].isin(to_drop)]
# We are going to recommend listings based of rest,safety.. but if there are no listings in that area we cannot reommend it..
# so here I am just going to drop Yonkers and New Rochelle,Floral Park from crime ..
match_hoods=match_hoods.dropna()

In [258]:
#Count records where there is no match on each side.
airbnb_nomatch = match_hoods.neighborhood_airbnb[match_hoods.neighborhood_crime.isnull()]
print('There are ' + str(airbnb_nomatch.sum()) + ' AirBnB listings with no match in the crime data.')

crime_nomatch = match_hoods.neighborhood_crime[match_hoods.neighborhood_airbnb.isnull()]
print('There are ' + str(crime_nomatch.sum()) + ' Crime listings with no match in the AirBnB data.')

There are 0.0 AirBnB listings with no match in the crime data.
There are 0.0 Crime listings with no match in the AirBnB data.


In [259]:
crime=crime[~(crime.neighborhood.isin(["Yonkers", "New Rochelle",'Floral Park ']))]

In [260]:
airbnb=airbnb[~(airbnb.neighborhood.isin(["Queens", "Bronx",'Brooklyn']))]

In [261]:
airbnb.neighborhood.nunique()

48

In [262]:
#crime_nhoods=crime[['CMPLNT_NUM','zipcode','Latitude','Longitude','crime_type']]

In [263]:
#crime_nhoods.dtypes

In [264]:
#crime_nhoods.head(2)

In [266]:
#a_hoods = airbnb_copy[['id_airbnb', 'latitude', 'longitude','zipcode']]
#a_hoods.head()

In [83]:
#my_airbnb=pd.merge(a_hoods,crime_nhoods,how='inner',on='neighborhood',suffixes = ('_airbnb', '_crime'))
# getting memory error when trying to merge on neighborhood lets try zipcode

In [37]:
# https://forums.fast.ai/t/memory-error-on-python-pandas-merge-for-large-databases/8407/2

In [267]:
import gc
gc.collect()

1635

In [268]:
#crime_nhoods.to_csv('crime_nhoods.csv')


In [271]:
#crime_nhoods.dtypes

In [272]:
#a_hoods.to_csv('a_hoods.csv')

In [273]:
'''
types_dict = {'CMPLNT_NUM': 'int8',
             'zipcode': 'int8',
             'Latitude': 'float16',
              'Longitude':'float16',
              'crime_type':'object'
             }'''

"\ntypes_dict = {'CMPLNT_NUM': 'int8',\n             'zipcode': 'int8',\n             'Latitude': 'float16',\n              'Longitude':'float16',\n              'crime_type':'object'\n             }"

In [274]:
#crime_nhoods=pd.read_csv('crime_nhoods.csv',low_memory=True,index_col=0,dtype=types_dict)

In [114]:
#crime_nhoods.dtypes

In [66]:
#crime_nhoods['zipcode']=crime_nhoods['zipcode'].astype(np.int64)

In [275]:

#types_dict1 = {'id_airbnb':'int8','latitude':'float16','longitude':'float16','zipcode':'int8'}
             

In [276]:
#a_hoods=pd.read_csv('a_hoods.csv',low_memory=True,index_col=0,dtype=types_dict1)

In [133]:
#a_hoods.head()

In [277]:
#a_hoods.dtypes

In [162]:
#my_airbnb=pd.merge(a_hoods,crime_nhoods,on='zipcode')

In [278]:
# https://www.kaggle.com/jagangupta/memory-optimization-and-eda-on-entire-dataset
mem_test=crime.memory_usage(index=True).sum()
print("crime dataset uses ",mem_test/ 1024**2," MB")
mem_test1=airbnb.memory_usage(index=True).sum()
print("airbnb dataset uses ",mem_test1/ 1024**2," MB")



crime dataset uses  494.99595642089844  MB
airbnb dataset uses  11.98681640625  MB


In [279]:
a_hoods.zipcode.nunique()

183

In [170]:
# lets try something else

In [280]:
crime.head(2)

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,LAW_CAT_CD,Latitude,Longitude,Lat_Lon,zipcode,neighborhood
0,876525802,BROOKLYN,04/10/2008,MISDEMEANOR,40.669,-73.913,"40.669,-73.913",11212.0,Central Brooklyn
1,402693528,MANHATTAN,06/03/2007,MISDEMEANOR,40.802,-73.946,"40.802,-73.946",10026.0,Central Harlem


In [281]:
#crime['crime_type']=crime['crime_type'].astype(int)

In [282]:
#pd.to_numeric(crime['crime_type'],errors='coerce')

In [283]:
crime = pd.get_dummies(crime, columns=['LAW_CAT_CD'])

In [284]:
crime.head()

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,Latitude,Longitude,Lat_Lon,zipcode,neighborhood,LAW_CAT_CD_FELONY,LAW_CAT_CD_MISDEMEANOR,LAW_CAT_CD_VIOLATION
0,876525802,BROOKLYN,04/10/2008,40.669,-73.913,"40.669,-73.913",11212.0,Central Brooklyn,0,1,0
1,402693528,MANHATTAN,06/03/2007,40.802,-73.946,"40.802,-73.946",10026.0,Central Harlem,0,1,0
2,832030069,QUEENS,02/16/2010,40.700,-73.831,"40.7,-73.831",11418.0,Southwest Queens,1,0,0
3,538308771,BROOKLYN,11/10/2009,40.681,-73.955,"40.681,-73.955",11216.0,Central Brooklyn,0,1,0
5,141085517,BROOKLYN,06/23/2011,40.678,-73.926,"40.678,-73.926",11233.0,Central Brooklyn,1,0,0


In [285]:
crime['score']=5*crime['LAW_CAT_CD_FELONY']+3*crime['LAW_CAT_CD_MISDEMEANOR']+2*crime['LAW_CAT_CD_VIOLATION']

In [286]:
df=crime.groupby('zipcode')['score'].mean().reset_index(name ='crime_score')

In [287]:
df.nlargest(columns='crime_score',n=5)

,zipcode,crime_score
115,11021.0,5.000000
117,11096.0,4.311111
50,10152.0,4.000000
55,10165.0,3.934051
110,10701.0,3.916667


In [288]:
df.to_csv('zipcodes_crime_score.csv')

In [289]:
my_dict = dict(zip(df['zipcode'], df['crime_score']))
airbnb['crime_score'] = airbnb['zipcode'].map(my_dict)

In [290]:
airbnb.head(2)

,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,...,restaurants_avg_rating_5,total_bars_1,total_bars_reviews_1,bars_avg_rating_1,bars_yelp_5,total_bars_reviews_5,bars_avg_rating_5,nearest_station,distance_to_station,crime_score
0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,...,4.057692,1.0,10.0,4.5,21.0,8701.0,4.000000,42nd St - Bryant Pk,0.093386,3.666079
1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,295,...,4.240741,NaN,NaN,NaN,12.0,790.0,4.208333,Classon Ave,0.417452,3.554833


In [291]:
airbnb.to_csv('Airbnb_Final_Merge.to_csv')

In [292]:
airbnb.crime_score.mean()

3.533510761851535

In [293]:
# the problem with this method is that it all the listings in the same zipcode will have same crime score!!

In [294]:
airbnb.zipcode.value_counts()

In [295]:
airbnb.isnull().sum()

id_airbnb                             0
host_id                               0
borough                               0
neighbourhood_group                   0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                     0
calculated_host_listings_count        0
availability_365                      0
description                           0
accommodates                          0
zipcode                               0
neighborhood                          0
total_restaurants_1               24795
total_restaurants_reviews_1       24795
restaurants_avg_rating_1          24795
restaurants_yelp_5                 2823
total_restaurants_reviews_5        2823
restaurants_avg_rating_5           2823
total_bars_1                      36531
total_bars_reviews_1              36531


In [11]:
airbnb=pd.read_csv('listings.csv')

C:\Apps\HarshD\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
airbnb.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20191204162729,2019-12-07,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","- Spacious (500+ft²), immaculate and nicely fu...","Beautiful, spacious skylit studio in the heart...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,1,1,0,0,0.39
1,3831,https://www.airbnb.com/rooms/3831,20191204162729,2019-12-07,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1,1,0,0,4.67


In [13]:
airbnb.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       ...
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object', length=106)

In [6]:
final=pd.read_csv('Airbnb_Final_Merge.to_csv')

In [7]:
final.head()

,Unnamed: 0,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,...,restaurants_avg_rating_5,total_bars_1,total_bars_reviews_1,bars_avg_rating_1,bars_yelp_5,total_bars_reviews_5,bars_avg_rating_5,nearest_station,distance_to_station,crime_score
0,0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,...,4.057692,1.0,10.0,4.5,21.0,8701.0,4.000000,42nd St - Bryant Pk,0.093386,3.666079
1,1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,...,4.240741,NaN,NaN,NaN,12.0,790.0,4.208333,Classon Ave,0.417452,3.554833
2,2,5099,7322,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,...,4.166667,NaN,NaN,NaN,6.0,2406.0,4.083333,Grand Central - 42nd St,0.427433,3.653282
3,3,5121,7356,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,29,...,3.970588,1.0,61.0,4.5,7.0,599.0,4.142857,Bedford - Nostrand Aves,0.368296,3.532187
4,4,5178,8967,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,...,4.037234,1.0,428.0,4.0,24.0,10378.0,4.083333,50th St,0.284923,3.581986


In [14]:
airbnb=airbnb[['id','number_of_reviews_ltm',
'review_scores_rating',                            
'review_scores_accuracy',                          
'review_scores_cleanliness',                       
'review_scores_checkin',                           
'review_scores_communication',                     
'review_scores_location',                          
'review_scores_value']]

In [16]:
final_data=pd.merge(final,airbnb,left_on='id_airbnb',right_on='id')

In [17]:
final_data.head()

,Unnamed: 0,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,...,crime_score,id,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,...,3.666079,2595,7,94.0,9.0,9.0,10.0,10.0,10.0,9.0
1,1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,...,3.554833,3831,75,90.0,9.0,9.0,10.0,9.0,10.0,9.0
2,2,5099,7322,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,...,3.653282,5099,8,90.0,10.0,9.0,10.0,10.0,10.0,9.0
3,3,5121,7356,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,29,...,3.532187,5121,0,90.0,8.0,8.0,10.0,10.0,9.0,9.0
4,4,5178,8967,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,...,3.581986,5178,47,84.0,9.0,7.0,9.0,9.0,10.0,8.0


In [19]:
final_data.drop(['id'],axis=1,inplace=True)

In [20]:
final_data.to_csv('Data-Project.csv')